### Introduction
This Workflow simulates putting the E. Coli core genome under the regulation of specific transcription factors and mutating certain reactions. The programm then calculates the changed metabolic flux. The changes can be visualized with an *escher* metabolic map.

### Set up symstem

In [1]:
# changed or new functions for this workflow
%run "TFMetEngSim.ipynb" 

print('System ready')

System ready


### List of reactions regulated by certain TFs
According to the aim of the experiment and the used medium you can choose the transcription factor(s) from this list.
_a stands for the activating properties and _r for the repressing properties of a TF.

[kurze Erklärung zu den einzelnen TF und wodurch (welches Medium) sie aktiviert werden]

In [ ]:
# for 'CRP_a'
#CRP_activated = ['PFL','PGK','ACALD','ACKr','PPCK','AKGDH','CS','SUCCt2_2','SUCDi','SUCOAS','FBA','FBP','FORt2','FORt','FRUpts2','FUM','FUMt2_2','GAPD','GLCpts','GLNS','MALt2_2','MDH','PDH']
CRP_activated = ['PFL','PGK','ACALD','ACKr','CS','SUCCt2_2','SUCDi','SUCOAS','FBA','FBP','FORt2','FORt','FRUpts2','FUM','FUMt2_2','GAPD','MALt2_2','MDH',] # only ones not also repressed
# for 'CRP_r'
CRP_repressed = ['PPCK','AKGDH','GLCpts','GLNS','GLUDy','GLUSy','ICL','MALS','PDH'] # all that can be repressed
#CRP_repressed = ['GLUDy','GLUSy','ICL','MALS'] # only the ones not also activated
# can both activate and repress itself
# Medium: df.loc[51, 'Fluxes']=0

# for 'Cra_a'
# Cra_activated = ['ACONTa','ACONTb','PPCK','PPS','CYTBD','FRUpts2','GLCpts','ICDHyr','ICL','MALS'] # all that can be activated
Cra_activated = ['PPCK','PPS','CYTBD','ICDHyr','ICL','MALS'] # only the ones not also repressed
# for 'Cra_r'
Cra_repressed = ['PFK','PGK','ALCD2x','PPC', 'ACONTa','ACONTb','PYK','ENO','TPI','FBA','FRUpts2','G6PDH2r','GAPD','GLCpts','PDH'] #all that can be repressed
Cra_repressed = ['PFK','PGK','ALCD2x','PPC','PYK','ENO','TPI','FBA','G6PDH2r','GAPD','PDH'] # only the ones not also acticated
# deactivated by fructose-1-phosphate and fructose-1,6-bisphosphate
# FRUpts2?
# df.loc[49, 'Fluxes']=0; ...

# for 'ArcA_a'
# ArcA_activated = ['PFL','ACKr','AKGDH','CYTBD','SUCDi','SUCOAS','FUM']
# ArcA_repressed = ['ACONTa','ACONTb','AKGDH','CS','SUCCt2_2','D_LACt2','SUCDi','SUCOAS','FUM','GLCpts','GLNS','ICDHyr','MALS','MALt2_2','MDH','NADH16','PDH']
# according to https://pubmed.ncbi.nlm.nih.gov/7855430/: AKGDH negative, SUCDi negative, SUCODAS negative, FUM eher positive

# for 'ArcA_a'
ArcA_activated = ['PFL','ACKr','CYTBD','FUM']
# for 'ArcA_r'
ArcA_repressed = ['ACONTa','ACONTb','AKGDH','CS','SUCCt2_2','D_LACt2','SUCDi','SUCOAS','GLCpts','GLNS','ICDHyr','MALS','MALt2_2','MDH','NADH16','PDH']
# higher concentration under aerobic conditions

# for 'FNR_a'
# FNR_activated = ['PFL','Plt2r','ALCD2x','ACKr','ACONTa','ACONTb','AKGDH','CYTBD','FRD7','FUM','GLUDy','NH4t','PDH']
# # for 'FNR_r'
# FNR_repressed = ['PFL','AKGDH','CYTBD','SUCDi','SUCOAS','FORt2','FORt','G6PDH2r','GLNS','GLUSy','GND','NADH16','NH4t','PDH'] 
# according to https://pubmed.ncbi.nlm.nih.gov/7855430/: PFL positive, AKGDH: ?, CYTBD negative,NH4t: ammonium transport, PDH: ?

#FNR_activated = ['PFL','Plt2r','ALCD2x','ACKr','ACONTa','ACONTb','AKGDH','FRD7','FUM','GLUDy','NH4t','PDH']
FNR_activated = ['PFL','Plt2r','ALCD2x','ACKr','ACONTa','ACONTb','FRD7','FUM','GLUDy'] # only not repressed ones
# for 'FNR_r'
FNR_repressed = ['AKGDH','CYTBD','SUCDi','SUCOAS','FORt2','FORt','G6PDH2r','GLNS','GLUSy','GND','NADH16','NH4t','PDH']
#FNR_repressed = ['CYTBD','SUCDi','SUCOAS','FORt2','FORt','G6PDH2r','GLNS','GLUSy','GND','NADH16'] # only not activated ones
# FNR generally represses itself
# active only under anaerobic conditions
# df.loc[59, 'Fluxes']=0

# for 'NarL_a'
NarL_activated = ['NADH16']
# for 'NArL_r'
NarL_repressed = ['PFL','ALCD2x','FORt2','FORt','FRD7','FUM']
# active while availability of nitrate during anaerobiosis 
# nitrate not in the model? anaerob: df.loc[59, 'Fluxes']=0

print('transcription factor targets loaded')

### Set up wildtype and add transcription factor regulation
Set the name of the transcriptionfactor as string on the second variable of `add_TranscriptionFactor()` and set the RctIDs of the targeted enzymes as list of strings as the third variable. Add more rows with more regulators if needed.

In [ ]:
# new unique promoters and their Expr stengths
# PromStenghts= {'GCCCATTGACAAGGCTCTCTCGGCCAGGTATAATTGCAAA': 5.133,
#                'GCCCATTGACAAGGCTCTCGCGCGAGTGTATAATTGCACG': 4.787,
#                'GCCCATTGACAAGGCTCTCGCGGCCAGGTATAATTGCAAA': 5.537, TAATTCCCC, 27.2%,
#                'GCCCATTGACAAGGCTCTCGTTACCAAGTATAATTGCAAA': 4.956,
#                'GCTTATTGACAAGGCTCTCGTTACCAAGTATAAGTGCGGG': 4.903
#                'GCCCATTGAGCCGAAGGTGGTCGAGCGATATAATTGCACG': 1.7}
 

In [2]:
wtHost = Ecol()
print('Wild type growth rate: {:.2f}'.format(wtHost.strain.objective))

Wild type growth rate: 0.87


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [3]:
# change the medium

medium = wtHost.strain.model.medium
medium["EX_o2_e"] = 0.0 # make anaerobic conditions
# medium["EX_glc__D_e"] = 0.0 # medium without glucose
# medium["EX_fru_e"] = 10.0 # medium with fructose
# medium["EX_no3_e"] = ... # medium with nitrate

wtHost.strain.model.medium = medium
wtHost.strain.model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

In [4]:
wtHost.strain.model.slim_optimize()

0.2116629497353107

In [5]:
print('Wild type growth rate: {:.2f}'.format(wtHost.strain.objective))
pd.set_option('display.max_rows', None) # show all rows
wtHost.strain.genes_df

Wild type growth rate: 0.87


,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux
0,PFK,1.749,GCCCATTGAGCCGCTTGAAATATTTTCCTAACATTGCACG,ATGTCACGCCACTAA,7.477382,4.275233
1,PFL,1.393,GCCCATTGAGATAATACAATCACAGTATTAGTGTTGCACG,ATGTGGAAGATCTGA,0.000000,0.000000
2,PGI,1.901,GCCCATTGAAGATCATGCCTCTATTAGATAGATTTGCACG,ATGTCCTGTCGTTGA,4.860861,2.557002
3,PGK,0.926,GCCCATTGAGCAGTATGTGTTCCGTAGGTAATATTGCACG,ATGATGATAATCTAG,-16.023526,-17.304024
4,PGL,2.466,GCCCATTGACCCTGAAGAGGCGGGTTCATAGTATTGCACG,ATGGCTGGTGGGTAA,4.959985,2.011348
5,ACALD,1.432,GCCCATTGAGACTGTATTATAGTATTATTAAGGTTGCACG,ATGAGTTTTGTGTGA,0.000000,0.000000
6,AKGt2r,1.214,GCCCATTGATGTGTAAATGGATTGCGGCTATTTTTGCACG,ATGTGGATTCTTTGA,0.000000,0.000000
7,PGM,3.016,GCCCATTGAAGGCCGAACTGCCAAAAGATAAACTTGCACG,ATGGTGGAGATATAA,-14.716140,-4.879357
8,PIt2r,1.590,GCCCATTGATTAATTATAACCCAGAGGCTAACATTGCACG,ATGGAAATTTGCTAA,3.214895,2.021947
9,ALCD2x,1.279,GCCCATTGATGTTGCACTCATTGTAAAGTACGATTGCACG,ATGACCACGAAATAA,0.000000,0.000000


In [ ]:
TFname1 = 'CRP_a'
TFname2 = 'CRP_r'
# adding regulator TF to dataframe and genome of wt
WTdf = wtHost.strain.genes_df
WTdf = add_TranscriptionFactor(WTdf,TFname1,'Activator','GCCCATTGAGCCGAAGGTGGTCGAGCGATATAATTGCACG','ATGCCCCCCAAGTAA')
WTdf = add_TranscriptionFactorReg(WTdf,TFname1,CRP_activated)
WTdf = add_TranscriptionFactor(WTdf,TFname2,'Repressor','GCCCATTGAGCCGAAGGTGGTCGAGCGATATAATTGCACC','ATGCCCCCCAATTAA')
WTdf = add_TranscriptionFactorReg(WTdf,TFname2,CRP_repressed)

# update wildtype dataframe's Expression and Expr2Flux
WTdf = Update_DF(WTdf)
# putting the genes dataframe back to the wild type
wtHost.strain.genes_df = WTdf
# Generating new Genome
WTGenome_new = Help_GenomeGenerator(WTdf, 500, .6)
wtHost.strain.genome = Seq(WTGenome_new)

wtHost.strain.genes_df

### Mutate the Host
Insert the RctIDs of the Reactions you want to mutate in `Mutants` as a list of strings. 
Insert the mutated Sequence in `insertSeq` as a string.
The sequence to be replaced is standardized as the always present ATTGA.

In [ ]:
# Generating mutant (define the mutations by hand and delete the automatically generated genome for now)

myHost = deepcopy(wtHost)

Mutants = [TFname2, TFname1]
Mutant_Targets = Mutants
insertSeq = 'ATTCCCCTGGAA'
# original: 'ATTGA'

# # mutate Promoter in Genome and genes_df
# # loop doesn't work in function, only last one mutated
# MutGenome, MTdf = Help_mutatePromoter(Mutants, wtHost.strain, myHost.strain, insertSeq, cutSeq='ATTGA')
# myHost.strain.genome = Seq(MutGenome)
# myHost.strain.genes_df = MTdf.copy()
for myMutant in Mutant_Targets:
#     Mutant_Bool = myHost.strain.genes_df['RctID']==myMutant
#     Mutant_Indx = tuple(np.arange(len(Mutant_Bool))[Mutant_Bool])
    MTdf = myHost.strain.genes_df.copy()
    Mutant_Indx = tuple(MTdf[MTdf['RctID'] == myMutant].index.values)
    # Finding the index for later printing
    WTIndx = str(wtHost.strain.genome).find(wtHost.strain.genes_df.loc[Mutant_Indx, 'Promoter'])
#     # Generating target promoter sequence, ATTGA is always there
    PromTar = MTdf.loc[Mutant_Indx, 'Promoter'].replace('ATTGA',insertSeq)
    # converting Biopython Seq class to string. This makes string replacements easier
    MutGenome = str(myHost.strain.genome)
    MutGenome = MutGenome.replace(MTdf.loc[Mutant_Indx, 'Promoter'], PromTar)
    myHost.strain.genome = Seq(MutGenome)
    MTdf.loc[Mutant_Indx, 'Promoter'] = PromTar
    myHost.strain.genes_df = MTdf.copy()
#     print('{}, Reference:\t{},\nMutated:\t{}\nwith index {}'.format(myMutant, wtHost.strain.genome[Mutant_Indx:Mutant_Indx+41], myHost.strain.genome[Mutant_Indx:Mutant_Indx+41], Mutant_Indx))
    print(myMutant, WTIndx)
    
# introducing a new Dataframe with old and new Expression and FLux, respectively
AllNewDF = measure_EnzymeLevel('Ecol', wtHost.strain, myHost.strain)

In [ ]:
# change for a TF with both activatiing and repressing property
'''because with more than one mutation, the NewExpr is calculated differently 
in comparison with the NewExpr with just one mutation, 
it is changed manually, so that it is the same TF'''
AllNewDF.loc[96, 'NewExpr'] = AllNewDF.loc[95, 'NewExpr']
AllNewDF['NewFlux'] = AllNewDF['NewExpr']*WTdf['Expr2Flux']

In [ ]:
# calculate the NewExpr of TF-regulated reactions and mutated reactions
CorrRegExpr = calculate_mutatedRegulatedExpression(WTdf, AllNewDF)
AllNewDF['NewExpr'] = CorrRegExpr
AllNewDF['NewFlux'] = AllNewDF['NewExpr']*WTdf['Expr2Flux']
AllNewDF

## Calculation of mutant flux distribution
The updated expression activities are converted to reference flux values and according to the reference flux new boundaries are set in the GSMM. The wild type and mutant flux distributions are stored in a dictionary for visualization in Escher.

In [ ]:
solution,_ = Help_FluxCalculator('Ecol', myHost.strain, AllNewDF)
RefFluxDict = WTdf.set_index('RctID')['Fluxes'].to_dict()
NewFluxDict = {a:b for a,b in zip(WTdf['RctID'].values, solution)}
AllFluxDict = [RefFluxDict, NewFluxDict]
# {WTdf['RctID'].tolist(): solution}

### Visualization of flux differences
Escher is used to visualize the flux differences on the central carbon metabolism map. Open the output-html file `Escher-Metabol-Map.html` by double click in the Jupyter Lab environment and `Trust` the file.

Then load the out.json file in Data -> load reaction data

In [ ]:
# name of the new file
filename = 'CRP_rvoll_glcx'

In [ ]:
# create json file of AllFluxDict

import json

flux_dictionary = AllFluxDict
with open(filename+'.json', 'w') as f:
    json.dump(flux_dictionary, f)
    
print('created json file')

### Additional tools

In [ ]:
# load to excel file
AllNewDF.to_excel(filename+'.xlsx')

In [ ]:
# for visualizing all rows of a dataframe:
pd.set_option('display.max_rows', None)
# insert the name of the Dataframe:
#AllNewDF

In [ ]:
# test the strength of a promoter
a = Help_PromoterStrength('Ecol', 'GCCCATTCCCCTGGAAGCCGAAGGTGGTCGAGCGATATAATTGCACG')
a

##### Combinations of promoters and mutations:

Promoter: GCCCATTGACAAGGCTCTCGCGGCCAGGTATAATTGCAAA

* 5.537 --> 1.507:  'TAATTCCCC'

Promoter: GCCCATTGAGCCGAAGGTGGTCGAGCGATATAATTGCACG

* 1.7 --> 3.419:  'ATTCCCCTGGAA'
